# PyPerceive to select and load Percept recordings 

## 0. Loading packages and functions, defining paths

In [1]:
# Importing Python and external packages


import os
import sys
import importlib
import json
from dataclasses import dataclass, field, fields
from itertools import compress
import csv
import pandas as pd
import numpy as np

import scipy
import matplotlib.pyplot as plt
from scipy import signal

import openpyxl
from openpyxl import Workbook, load_workbook
import xlrd

#mne
import mne_bids
import mne
from mne.time_frequency import tfr_morlet 



from importlib import reload          

# from scipy.signal import spectrogram, hanning     # hanning not found


#### check package versions

developed with:
- Python sys 3.10.8
- pandas 1.5.1
- numpy 1.23.4
- mne_bids 0.11.1
- mne 1.2.3

In [2]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('mne_bids', mne_bids.__version__)
print('mne', mne.__version__)


Python sys 3.10.8 | packaged by conda-forge | (main, Nov 24 2022, 14:07:00) [MSC v.1916 64 bit (AMD64)]
pandas 1.5.1
numpy 1.23.4
mne_bids 0.11.1
mne 1.2.3


#### Load pyPerceive functions

In [3]:
def add_and_set_code_folder_in_notebook():
    """
    while working in the local pyPerceive repo,
    find and set path to the PyPerceive code folder

    use function in notebook first, to locate the local
    repo and enable import of pyPerceive functions
    """
    project_path = os.getcwd()

    while project_path[-10:] != 'PyPerceive':
        project_path = os.path.dirname(project_path)

    code_path = os.path.join(project_path, 'code')
    sys.path.append(code_path)

    # change directory to code path
    os.chdir(code_path)
    
    return print(f'working dir set to: {code_path}')


## 1. Load Data using MainClass

In [4]:
# change working directory to ensure correct loading of own functions
add_and_set_code_folder_in_notebook()

from PerceiveImport.classes import (
    main_class, modality_class, metadata_class,
    session_class, condition_class, task_class,
    contact_class, run_class, chronic_class
)
import PerceiveImport.methods.load_rawfile as load_rawfile
import PerceiveImport.methods.find_folders as find_folders
import PerceiveImport.methods.metadata_helpers as metaHelpers



working dir set to: c:\Users\habetsj\Research\projects\PyPerceive\code


In [5]:
# reload classes during debugging
importlib.reload(main_class)
importlib.reload(modality_class)
importlib.reload(session_class)
importlib.reload(task_class)
importlib.reload(condition_class)
importlib.reload(contact_class)
importlib.reload(metadata_class)
importlib.reload(load_rawfile)
importlib.reload(find_folders)
importlib.reload(run_class)
importlib.reload(metaHelpers)

<module 'PerceiveImport.methods.metadata_helpers' from 'c:\\Users\\habetsj\\Research\\projects\\PyPerceive\\code\\PerceiveImport\\methods\\metadata_helpers.py'>

In [6]:
# define an example instance and fill in the values of the dataclass PerceiveData 
# choose the values you are interested in analyzing further

sub024 = main_class.PerceiveData(
    sub = "024", 
    incl_modalities=['survey', 'streaming', 'indefiniteStreaming'],
    incl_session = ["fu18m"],
    incl_condition =['m0s0'],
    incl_task = ["rest"],
    incl_contact = ["RingL", "SegmInterR", "SegmIntraR"],
    import_json=False,
    warn_for_metaNaNs=True,
    # use_bids=True,  # TODO: add to functionality
)



	### WARNING: NaNs in Metadata Table sub-024 ###
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618063700.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064000.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064200.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084000.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084300.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084700.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921070500.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921072300.mat
NaNs in: sub024_ses-2021092106385396_run-CHRONIC20210917114914.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921074800.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921080600.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921082400.mat
NaNs in: sub024_ses-2021092107452096_run-CHRONIC20210918121914.mat
NaNs in: sub-20210615PStn_ses-2022061010445782

In [11]:
sub024.indefiniteStreaming.fu18m.m0s0.rest.run1.data.ch_names

['LFP_L_03_STN_MT',
 'LFP_L_13_STN_MT',
 'LFP_L_02_STN_MT',
 'LFP_R_03_STN_MT',
 'LFP_R_13_STN_MT',
 'LFP_R_02_STN_MT',
 'LFP_L_01_STN_MT',
 'LFP_L_12_STN_MT',
 'LFP_L_23_STN_MT',
 'LFP_R_01_STN_MT',
 'LFP_R_12_STN_MT',
 'LFP_R_23_STN_MT']

In [35]:
sub024.streaming.fu18m.m0s0.rest.run1.data

<RawArray | 6 x 30187 (120.7 s), ~1.4 MB, data loaded>

In [36]:
sub024.indefiniteStreaming.fu18m.m0s0.rest.run1.data

<RawArray | 12 x 6500 (26.0 s), ~626 kB, data loaded>

In [6]:
# define an example instance and fill in the values of the dataclass PerceiveData 
# choose the values you are interested in analyzing further
importlib.reload(main_class)
importlib.reload(modality_class)
importlib.reload(chronic_class)

chronic24 = main_class.PerceiveData(
    sub = "024", 
    incl_modalities=['chronic'],
    incl_task = ["rest"],
    import_json=True,
    warn_for_metaNaNs=False,
    # use_bids=True,  # TODO: add to functionality
)


sub024_ses-2021092106385396_run-CHRONIC20210917114914.mat FAILED
Report_Json_Session_Report_20210921T120859.json
sub024_ses-2021092107452096_run-CHRONIC20210918121914.mat FAILED
Report_Json_Session_Report_20210921T120956.json
sub-20210615PStn_ses-2022061010445782_run-CHRONIC20210918012914.mat FAILED
Report_Json_Session_Report_20220613T133239.json
sub-20210615PStn_ses-2022072210000082_run-CHRONIC20210918125914.mat FAILED
Report_Json_Session_Report_20220613T133123.json
sub-20210615PStn_ses-1948072210000072_run-CHRONIC20210918022914.mat FAILED
Report_Json_Session_Report_20221205T134700.json


In [69]:
jsonfile = 'Report_Json_Session_Report_20210921T120859.json'
dat = load_rawfile.load_sourceJSON('024', jsonfile)

In [10]:
dat.keys()
# dat.hdr.Groups.Initial(b).ProgramSettings.SensingChannel(2).SensingSetup.FrequencyInHertz

dict_keys(['AbnormalEnd', 'FullyReadForSession', 'FeatureInformationCode', 'SessionDate', 'SessionEndDate', 'ProgrammerTimezone', 'ProgrammerUtcOffset', 'ProgrammerLocale', 'ProgrammerVersion', 'PatientInformation', 'DeviceInformation', 'BatteryInformation', 'LeadConfiguration', 'Stimulation', 'Groups', 'BatteryReminder', 'MostRecentInSessionSignalCheck', 'Impedance', 'GroupHistory', 'SenseChannelTests', 'CalibrationTests', 'LfpMontageTimeDomain', 'BrainSenseTimeDomain', 'BrainSenseLfp', 'LFPMontage', 'EventSummary', 'DiagnosticData'])

In [16]:
dat['DiagnosticData'].keys()

dict_keys(['LFPTrendLogs', 'EventLogs'])

In [148]:
dat['Groups']['Initial'][0]['ProgramSettings']['SensingChannel'][0].keys()

# print(
#     dat['Groups']['Initial'][0]['ProgramSettings']['SensingChannel'][0]['Channel'],
#     dat['Groups']['Initial'][0]['ProgramSettings']['SensingChannel'][0]['UpperLfpThreshold'],
#     dat['Groups']['Initial'][0]['ProgramSettings']['SensingChannel'][0]['LowerLfpThreshold']
# )

# dat['Groups']['Initial'][0]['ProgramSettings']['SensingChannel'][0]['SensingSetup']['FrequencyInHertz']

# chSig = dat['Groups']['Initial'][0]['ProgramSettings']['SensingChannel'][0]['SensingSetup']['ChannelSignalResult']
# for k in chSig: print(k, chSig[k])
groups = dat['Groups']['Initial']  # groups[0]['GroupId'] -> 'GroupIdDef.GROUP_A' 
sense_channels = groups[0]['ProgramSettings']['SensingChannel']
freq = sense_channels[0]['SensingSetup']['FrequencyInHertz']

freq

for n in np.arange(len(groups)):

    if groups[n]['ActiveGroup']:
        print(f'{groups[n]["GroupId"]} is active group')
        if 'SensingChannel' in groups[n]['ProgramSettings'].keys():
            print(groups[n]['ProgramSettings']['SensingChannel'])
        else:#
            print('\tNo SensingChannel in active group')

GroupIdDef.GROUP_B is active group
	No SensingChannel in active group


In [149]:
dat['DiagnosticData']['LFPTrendLogs']['HemisphereLocationDef.Left']

{'2021-09-18T06:44:36Z': [{'DateTime': '2021-09-18T04:59:15Z',
   'LFP': 442,
   'AmplitudeInMilliAmps': 1.9},
  {'DateTime': '2021-09-18T04:49:15Z',
   'LFP': 357,
   'AmplitudeInMilliAmps': 1.9},
  {'DateTime': '2021-09-18T04:39:15Z',
   'LFP': 296,
   'AmplitudeInMilliAmps': 1.9},
  {'DateTime': '2021-09-18T04:29:15Z',
   'LFP': 292,
   'AmplitudeInMilliAmps': 1.9},
  {'DateTime': '2021-09-18T04:19:15Z',
   'LFP': 282,
   'AmplitudeInMilliAmps': 1.9},
  {'DateTime': '2021-09-18T04:09:15Z',
   'LFP': 331,
   'AmplitudeInMilliAmps': 1.9},
  {'DateTime': '2021-09-18T03:59:15Z',
   'LFP': 429,
   'AmplitudeInMilliAmps': 1.9},
  {'DateTime': '2021-09-18T03:49:15Z',
   'LFP': 476,
   'AmplitudeInMilliAmps': 1.9},
  {'DateTime': '2021-09-18T03:39:15Z',
   'LFP': 277,
   'AmplitudeInMilliAmps': 1.9},
  {'DateTime': '2021-09-18T03:29:15Z',
   'LFP': 222,
   'AmplitudeInMilliAmps': 1.9},
  {'DateTime': '2021-09-18T03:19:15Z',
   'LFP': 225,
   'AmplitudeInMilliAmps': 1.9},
  {'DateTime': '202

In [36]:
mat = scipy.io.loadmat(
    os.path.join(find_folders.get_onedrive_path('sourcedata', sub='024'),
                 'sub024_ses-2021092106385396_run-CHRONIC20210917114914.mat'
    )
)

In [68]:
mat['data'][0][0][1][0]

'DiagnosticData.LFPTrends'

In [342]:
# if jsons are imported
# json_object=sub024.survey.fu18m.m0s0.rest.RingR.json
# json_object["RingR"]["Impedance"]